Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

In [11]:
#imports
import os
import requests
import json
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display

In [2]:
%pip install urllib

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement urllib (from versions: none)
ERROR: No matching distribution found for urllib


In [3]:
#uncomment to import the library based on the provider you prefer to use.

# import ollama
# from openai import OpenAI
from groq import Groq

In [5]:
# uncomment the model you prefer to use for your tool

# MODEL_LLAMA = "llama3.2"
# MODEL_GPT = "gpt-4o-mini"
MODEL_GROQ_DEEPSEEK = "deepseek-r1-distill-llama-70b"

In [10]:
# if using gpt or groq, load api key, else ignore if using ollama
load_dotenv(override=True)
api_key = os.getenv("GROQ_API_KEY") # change env variable name to the one present in your .env file

client = Groq(api_key=api_key)
# client = OpenAI(api_key=api_key)
# client = ollama

In [7]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

Use a call to the LLM model to read the links on a webpage, and respond in structured JSON.

It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".

We will use "<b>one shot prompting</b>" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

In [8]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
    You can decide which of the links would be most relevant to be included in a brochure about the company, \
        such as links to an About page, a Company page, or Careers/Jobs pages. \n"

link_system_prompt += "You must respond in JSON as shown in this example:\n"
link_system_prompt += """
    {
        "links": [
            {"type": "about page", "url": "https://url/about"},
            {"type": "careers page", "url": "https://url/relevant paths/careers"},
        ]
    }
"""
link_system_prompt += "Each 'url' key in the above JSON should be a single URL string and not a list of URL links"

In [9]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "Please decide which of these are relevant web links for a brochure about the company, and respond with the full \
        company HTTP URL in JSON format. Do not include links/pages such as Terms of Service, Privacy, Email, Courses, Invalid, Page \
            not found and positively make sure the company links you respond should all be valid and working links \n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [12]:
def get_links(url):
    website = Website(url)
    response = client.chat.completions.create(
        model = MODEL_GROQ_DEEPSEEK,
        messages = [
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
        ],
        temperature = 0.6,
        response_format = {"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [19]:
# Assemble all the details into another prompt to the LLM
def get_all_details(url):
    result = "Landing page: \n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found Links: ", links)
    for link in links["links"]:
        try:
            result += f"\n\n{link['type']}\n"
            result += Website(link["url"]).get_contents()
        except Exception as e:
            print("Error occured while getting contents of the website: " + link["url"])
            continue
    return result

In [15]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [16]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; \
        use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [17]:
def create_brochure(company_name, url):
    response = client.chat.completions.create(
        model = MODEL_GROQ_DEEPSEEK,
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
        temperature = 0.6
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [20]:
create_brochure("Cyfrin Updraft", "https://www.cyfrin.io/updraft")

Found Links:  {'links': [{'type': 'home page', 'url': 'https://www.cyfrin.io/'}, {'type': 'about page', 'url': 'https://www.cyfrin.io/updraft'}, {'type': 'services page', 'url': 'https://www.cyfrin.io/smart-contract-security-audits'}, {'type': 'case studies page', 'url': 'https://www.cyfrin.io/case-studies'}, {'type': 'blog page', 'url': 'https://www.cyfrin.io/blog'}, {'type': 'certifications page', 'url': 'https://www.cyfrin.io/certifications/solidity-certification'}, {'type': 'social media - twitter', 'url': 'https://twitter.com/CyfrinAudits'}, {'type': 'social media - discord', 'url': 'https://discord.gg/cyfrin'}, {'type': 'social media - youtube', 'url': 'https://www.youtube.com/@CyfrinAudits'}, {'type': 'social media - github', 'url': 'https://github.com/Cyfrin'}, {'type': 'social media - linkedin', 'url': 'https://www.linkedin.com/company/cyfrin/'}]}


<think>
Okay, I need to create a short brochure for Cyfrin Updraft based on the provided information. Let me start by reading through all the content to understand what the company does.

The landing page mentions that Cyfrin Updraft offers blockchain and smart contract development courses. They have a variety of services like private audits, competitions, education, certifications, research platforms, bug bounties, MPA, video tutorials, and documentation. They support multiple blockchain chains such as Ethereum, ZKsync, Arbitrum, etc. They also have case studies, resources like blogs, glossaries, and a brand kit. The careers section says they're hiring and always looking for talented people.

Looking at the "What is Cyfrin Updraft?" section, it's a free web3 education platform designed to help developers become blockchain engineers and security auditors. They've taught over 200,000 students, with a million lessons watched, 60k monthly active students, and 100+ hours of content. Their courses range from beginner to expert levels, focusing on smart contract development with step-by-step tutorials and real-world projects. The courses are free and accessible without signup.

The brochure needs to include company culture, customers, and careers. From the information, I can infer that the company values education, accessibility, and community. They seem to foster a culture of learning and growth. Their customers are likely developers, both new and experienced, looking to enter or advance in web3. Careers are available in various fields, and they emphasize being a great place to work with talented individuals.

I should structure the brochure with sections like About Us, Our Mission, Services, Customers, Culture, Careers, and a Call to Action. I'll need to highlight their impact, the range of services, the supportive culture, and the opportunities for growth.

I'll make sure to use markdown formatting, include links where necessary, and keep it concise and engaging for prospective customers, investors, and recruits.
</think>

# Cyfrin Updraft: Empowering Web3 Through Education

## About Us

Cyfrin Updraft is a leading provider of free web3 education, dedicated to helping developers kickstart and advance their careers in blockchain and smart contract development. Our platform offers comprehensive courses, certifications, and resources to foster growth and innovation in the web3 space.

## Our Mission

We believe in making high-quality education accessible to everyone. Our mission is to bridge the skills gap in the web3 industry by providing free, world-class training to aspiring blockchain engineers and security auditors.

## Services

- **Private Audits**: Enhance your smart contract security with our expert services.
- **Competitions**: Participate in hackathons and bug bounties to win prizes and gain recognition.
- **Education & Certifications**: Access a wide range of courses and certifications to validate your skills.
- **Research Platform**: Utilize our tools for advanced security research.
- **Community Resources**: Explore our blog, glossary, and documentation for the latest insights and knowledge.

## Our Impact

- **200,000+ Students Trained**: Empowering developers worldwide to enter the web3 industry.
- **1,000,000+ Lessons Watched**: Engaging content that drives learning and growth.
- **60,000+ Monthly Active Learners**: A vibrant community dedicated to continuous development.

## Customers

Our platform serves developers at all levels, from beginners exploring blockchain basics to seasoned experts refining their skills. Whether you're starting fresh or looking to advance, our resources are tailored to meet your needs.

## Company Culture

At Cyfrin Updraft, we value:

- **Accessibility**: Free education for all, regardless of background.
- **Innovation**: Constantly evolving to meet the latest industry demands.
- **Community**: A supportive environment for learning and collaboration.
- **Growth**: Encouraging continuous development and career advancement.

## Careers

Join our team of passionate innovators and educators. We are committed to fostering a culture of growth, collaboration, and impact. Explore our current openings and become part of a team shaping the future of web3.

## Join Us

Start your journey with Cyfrin Updraft today. Whether you're a student, a professional, or an investor, we invite you to be part of our mission to empower the next generation of web3 leaders.

---

**[Visit Cyfrin Updraft](https://cyfrinupdraft.com)** to learn more about our courses, services, and career opportunities. Together, let's build the future of blockchain.

With a small adjustment, we can change this so that the results stream back from OpenAI, with the familiar typewriter animation

In [21]:
def stream_brochure(company_name, url):
    stream = client.chat.completions.create(
        model = MODEL_GROQ_DEEPSEEK,
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        temperature = 0.6,
        stream = True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [22]:
stream_brochure("Cyfrin Updraft", "https://www.cyfrin.io/updraft")

Found Links:  {'links': [{'type': 'about page', 'url': 'https://www.cyfrin.io/updraft'}, {'type': 'case studies', 'url': 'https://www.cyfrin.io/case-studies'}, {'type': 'smart contract security audits', 'url': 'https://www.cyfrin.io/smart-contract-security-audits'}, {'type': 'certifications', 'url': 'https://www.cyfrin.io/certifications/solidity-certification'}, {'type': 'codehawks competitive audits', 'url': 'https://www.cyfrin.io/codehawks/competitive-audits'}, {'type': 'codehawks first flights', 'url': 'https://www.cyfrin.io/codehawks/first-flights'}, {'type': 'Twitter', 'url': 'https://twitter.com/CyfrinAudits'}, {'type': 'Discord', 'url': 'https://discord.gg/cyfrin'}, {'type': 'YouTube', 'url': 'https://www.youtube.com/@CyfrinAudits'}, {'type': 'GitHub', 'url': 'https://github.com/Cyfrin'}, {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/cyfrin/'}, {'type': 'success stories', 'url': 'https://www.cyfrin.io/success-stories'}, {'type': 'blog', 'url': 'https://www.cyfrin

<think>
Okay, so I need to create a short brochure for Cyfrin Updraft based on the provided landing page and other relevant pages. Let me start by understanding what the company does.

From the landing page, it seems Cyfrin Updraft is primarily an educational platform focused on blockchain and smart contract development. They offer free courses, certifications, and have a strong emphasis on community and career development. They also have a research platform and bug bounty programs, which suggests they're involved in the security aspect of blockchain technology.

Looking at the structure, the brochure should include sections like About Us, Our Offerings, Customers, Culture, Careers, and a Call to Action. Maybe also include some testimonials or success stories if available, but I don't see specific ones here.

Under About Us, I should highlight their mission to make web3 education accessible, their reach (like the number of students and lessons watched), and their role in both education and security.

Our Offerings should detail the courses, certifications, the research platform, and bug bounties. It's important to mention that the courses are free and open, which is a big selling point.

For Customers, the case studies mentioned are about companies like Sablier and Oku Trade, so I can note that they've helped secure DeFi platforms and others in the blockchain space. Their customers are likely businesses in the web3 and DeFi sectors.

Company Culture seems to focus on accessibility, community, and empowerment. They emphasize that education should be free, so that's a key cultural aspect. Also, their involvement in competitions and hackathons suggests a dynamic and engaging environment.

Careers would include the fact that they're hiring and value talented individuals who are passionate about blockchain. It might also mention the collaborative and innovative work environment.

I should also include a section on Getting Started, encouraging prospective students or users to take the first steps, maybe with a link or a call to action.

Finally, the brochure should end with a strong call to action, encouraging people to join, whether as students, customers, or employees.

I need to make sure the language is engaging and highlights the unique aspects of Cyfrin Updraft, like the free education, industry recognition, and the impact they've had on their students' careers.

I should also structure it in  with appropriate headings and sections, keeping it concise but informative. Maybe include some stats like the number of students and lessons watched to add credibility.

I think that's a good start. Now, I'll organize all these thoughts into the brochure structure.
</think>

# Welcome to Cyfrin Updraft

## About Us

Cyfrin Updraft is a pioneering educational platform dedicated to making web3 education accessible to everyone. Our mission is to empower the next generation of blockchain developers and security auditors by providing high-quality, free courses. With over 200,000 students and 1,000,000 lessons watched, we are a leader in the blockchain education space. Beyond education, we are committed to enhancing the security of the blockchain ecosystem through our research platform and bug bounty programs.

## Our Offerings

- **Free Courses**: Explore our extensive library of blockchain and smart contract development courses, designed for both beginners and experts. Learn from industry-recognized experts and gain hands-on experience through real-world projects.
  
- **Certifications**: Validate your skills with our industry-recognized certifications, helping you stand out in the competitive web3 job market.

- **Research Platform**: Access cutting-edge tools to stay ahead in blockchain security and development.

- **Bug Bounties**: Participate in our programs to test your skills and contribute to the security of the blockchain ecosystem.

## Our Customers

We have successfully supported numerous businesses in securing their platforms, including DeFi leaders like Sablier and Oku Trade. Our expertise in smart contract security has made us a trusted partner in the web3 and DeFi sectors.

## Our Culture

At Cyfrin Updraft, we believe in accessibility and community. We are committed to making education free and open to all, fostering a culture of empowerment and collaboration. Our engaging environment, including competitions and hackathons, encourages innovation and growth.

## Careers

Join our dynamic team of passionate blockchain enthusiasts. We value talent and innovation, offering a collaborative and forward-thinking work environment. Whether you're in education, security, or another field, we invite you to be part of our mission to shape the future of web3.

## Getting Started

Ready to embark on your web3 journey? Start with our foundational courses, explore advanced features, and engage with our community. Visit our platform to begin your learning journey today.

## Call to Action

**Join Cyfrin Updraft** and become part of a community that is shaping the future of blockchain technology. Whether you're a student, a professional, or an organization, we invite you to explore, learn, and grow with us.

---

**Cyfrin Updraft** - Empowering the Future of Blockchain